# Report handler examples

Install leakpro as ``` pip install -e /path/to/leakpro ```

### Synthetic examples

In [1]:
import os
import sys

import pandas as pd

sys.path.append("../..")

from leakpro.synthetic_data_attacks.anomalies import return_anomalies
from leakpro.synthetic_data_attacks.inference_utils import inference_risk_evaluation
from leakpro.synthetic_data_attacks.linkability_utils import linkability_risk_evaluation
from leakpro.synthetic_data_attacks.singling_out_utils import singling_out_risk_evaluation

#Get ori and syn
n_samples = 100
DATA_PATH = "../synthetic_data/datasets/"
ori = pd.read_csv(os.path.join(DATA_PATH, "adults_ori.csv"), nrows=n_samples)
syn = pd.read_csv(os.path.join(DATA_PATH, "adults_syn.csv"), nrows=n_samples)

In [2]:
syn_anom = return_anomalies(df=syn, n_estimators=1000, n_jobs=-1, verbose=True)
print("Syn anom shape",syn_anom.shape)

[Parallel(n_jobs=64)]: Using backend ThreadingBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done   2 out of  64 | elapsed:    1.2s remaining:   38.1s
[Parallel(n_jobs=64)]: Done  64 out of  64 | elapsed:    4.4s finished


Unique predictions (array([-1,  1]), array([ 3, 97]))
Syn anom shape (3, 14)


In [3]:
# Create a singling-out result
sin_out_res = singling_out_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_attacks = syn_anom.shape[0]
)

In [4]:
# Create linkability result
link_res = linkability_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_samples = syn_anom.shape[0],
    n_attacks = 100
)

In [5]:
# # Create base-case inference result
inf_res = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = False,
    n_attacks = syn_anom.shape[0]
)

# # Create worst-case inference result
inf_res_worst = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = True,
    n_attacks = syn_anom.shape[0]
)

### Gradient inversion example

In [6]:
from gia_utils.cifar import get_cifar10_loader
from gia_utils.model import ResNet
from torchvision.models.resnet import BasicBlock

from leakpro.attacks.gia_attacks.invertinggradients import InvertingConfig
from leakpro.fl_utils.gia_train import train
from leakpro.run import run_inverting

model = ResNet(BasicBlock, [5, 5, 5], num_classes=10, base_width=16 * 10)
client_dataloader, data_mean, data_std = get_cifar10_loader(num_images=1, batch_size=1, num_workers=2)

# Meta train function designed to work with GIA
train_fn = train

# Baseline config
configs = InvertingConfig()
configs.at_iterations = 80 # Decreased from 8000 to avoid GPU memory crash

name = "my_gia_results"
GIA_result = run_inverting(model, client_dataloader, train_fn, data_mean, data_std, configs, experiment_name=name, save=True)

Files already downloaded and verified


2024-12-10 15:37:19,407 INFO     Inverting gradient initialized.
2024-12-10 15:37:21,549 INFO     Iteration 0, loss 0.00017312286945525557
2024-12-10 15:37:21,558 INFO     New best loss: 0.00017312286945525557 on round: 0
2024-12-10 15:37:21,716 INFO     New best loss: 0.00015130748215597123 on round: 1
2024-12-10 15:37:21,865 INFO     New best loss: 0.00014282172196544707 on round: 2
2024-12-10 15:37:22,004 INFO     New best loss: 0.0001371056423522532 on round: 3
2024-12-10 15:37:22,146 INFO     New best loss: 0.00013112256419844925 on round: 4
2024-12-10 15:37:22,404 INFO     New best loss: 0.00012842075375374407 on round: 6
2024-12-10 15:37:22,531 INFO     New best loss: 0.00012392438657116145 on round: 7
2024-12-10 15:37:22,661 INFO     New best loss: 0.00011872354662045836 on round: 8
2024-12-10 15:37:22,915 INFO     New best loss: 0.00011686021025525406 on round: 10
2024-12-10 15:37:23,045 INFO     New best loss: 0.00011540275590959936 on round: 11
2024-12-10 15:37:23,176 INFO  

### Membership Inference Attack, CIFAR example

In [7]:
import os
import sys
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

In [8]:
from mia_utils.utils.cifar_data_preparation import get_cifar_dataloader
from mia_utils.utils.cifar_model_preparation import ResNet18, create_trained_model_and_metadata


# Load the config.yaml file
with open('mia_utils/train_config.yaml', 'r') as file:
    train_config = yaml.safe_load(file)

# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), train_config["data"]["data_dir"])

train_loader, test_loader = get_cifar_dataloader(path, train_config)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# Train the model
if not os.path.exists("target"):
    os.makedirs("target")
if train_config["data"]["dataset"] == "cifar10":
    num_classes = 10
elif train_config["data"]["dataset"] == "cifar100":
    num_classes = 100
else:
    raise ValueError("Invalid dataset name")

model = ResNet18(num_classes = num_classes)
train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
                                                                               train_loader, 
                                                                               test_loader, 
                                                                               train_config)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Training Progress: 100%|██████████| 3/3 [00:13<00:00,  4.60s/it]


##### Run the MIA attack

In [10]:
from mia_utils.cifar_handler import CifarInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "mia_utils/audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(CifarInputHandler, config_path)

# Run the audit 
mia_results = leakpro.run_audit(return_results=True)

2024-12-10 15:37:48,180 INFO     Target model blueprint created from ResNet18 in ./mia_utils/utils/cifar_model_preparation.py.
2024-12-10 15:37:48,183 INFO     Loaded target model metadata from ./target/model_metadata.pkl


{'audit': {'random_seed': 1234, 'attack_list': {'population': {'attack_data_fraction': 1.0}}, 'output_dir': './leakpro_output', 'attack_type': 'mia', 'modality': 'image'}, 'target': {'module_path': './mia_utils/utils/cifar_model_preparation.py', 'model_class': 'ResNet18', 'target_folder': './target', 'data_path': './data/cifar10.pkl'}, 'shadow_model': None, 'distillation_model': None}


2024-12-10 15:37:48,394 INFO     Loaded target model from ./target
2024-12-10 15:37:49,289 INFO     Loaded population dataset from ./data/cifar10.pkl
2024-12-10 15:37:49,290 INFO     Loaded population dataset from ./data/cifar10.pkl
2024-12-10 15:37:49,291 INFO     Creating shadow model handler singleton
2024-12-10 15:37:49,294 INFO     Creating distillation model handler singleton
2024-12-10 15:37:49,296 INFO     Configuring the Population attack
2024-12-10 15:37:49,297 INFO     Added attack: population
2024-12-10 15:37:49,298 INFO     Preparing attack: population
2024-12-10 15:37:49,299 INFO     Preparing attack data for training the Population attack
2024-12-10 15:37:49,306 INFO     Subsampling attack data from 24000 points
2024-12-10 15:37:49,307 INFO     Number of attack data points after subsampling: 24000
2024-12-10 15:37:49,308 INFO     Computing signals for the Population attack
Getting loss for model 1/ 1: 100%|██████████| 750/750 [00:12<00:00, 60.59it/s]
2024-12-10 15:38:01,

In [11]:
import sys
sys.path.append("../..")

# Import and initialize ReportHandler
from leakpro.reporting.report_handler import ReportHandler

# Set report_dir to "./leakpro_output/results" to the results to a local results folder
#    or don't use the report_dir argument to let the ReportHandler find an already
#    existing results folder
report_handler = ReportHandler(report_dir="./leakpro_output/results")

# # Save Synthetic results using the ReportHandler
report_handler.save_results(attack_name="singling_out", result_data=sin_out_res)
report_handler.save_results(attack_name="linkability_risk", result_data=link_res)
report_handler.save_results(attack_name="inference_risk_base", result_data=inf_res)
report_handler.save_results(attack_name="inference_risk_worst", result_data=inf_res_worst)

# # Save GIA results using report handler
report_handler.save_results(attack_name="gia", result_data=GIA_result)

# Save MIA resuls using report handler
for res in mia_results:
    report_handler.save_results(attack_name=res.attack_name, result_data=res, config=res.configs)

2024-12-10 15:38:19,719 INFO     Initializing report handler...
2024-12-10 15:38:19,719 INFO     report_dir set to: ./leakpro_output/results
2024-12-10 15:38:19,720 INFO     Saving results for singling_out
2024-12-10 15:38:21,860 INFO     Saving results for linkability_risk
2024-12-10 15:38:23,763 INFO     Saving results for inference_risk_base
2024-12-10 15:38:28,190 INFO     Saving results for inference_risk_worst
2024-12-10 15:38:30,418 INFO     Saving results for gia
2024-12-10 15:38:30,431 INFO     Saving results for population


<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

In [12]:
# Use the ReportHandler and load all the saved results
report_handler.load_results()

# Create results and collect corresponding latex texts
report_handler.create_results_all()

# Create the report by compiling the latex text
report_handler.create_report()

2024-12-10 15:38:59,171 INFO     PDF compiled


<Figure size 1920x1440 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x2100 with 0 Axes>

<Figure size 3300x1200 with 0 Axes>